# Gaussian Elimination and LU Decomposition


## Forward and Backward Substitution
Still start from the linear system

$$\textbf{Ax}=\textbf{b}$$

How can we use our computer to solve for it?

First, let's consider some matrix with the structure of upper triangular matrix and lower triangular matrix:

$$
U = \begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
0      & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0      & 0      & \cdots & a_{nn}
\end{bmatrix}
$$

$$
L = \begin{bmatrix}
a_{11} & 0      & \cdots & 0      \\
a_{21} & a_{22} & \cdots & 0      \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{bmatrix}
$$

This seems to be easy to solve, we can do that by using backward and forward substitution.

In [48]:
import numpy as np

def forward_substitution(A, b):
    '''
    Forward Substitution
    Input: Lower triangular, full-ranked, squared matrix A, and the RHS b
    Output: Solution to this system x
    '''
    # Initialize and copy the identity
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)
    n = A.shape[0]
    m = A.shape[1]
    x = np.zeros(n)

    # Check if it is lower triangular, full-ranked, squared
    A_lower_tri = np.tril(A, k=0)
    if not(np.equal(A, A_lower_tri).all()) or (np.diag(A) == 0).all() or m != n:
        raise ValueError("The input matrix is not a lower triangular, full-ranked, squared matrix.")
    
    for i in range(n):
        for j in range(i):
            b[i] = b[i] - A[i, j] * x[j]
        x[i] = b[i] / A[i, i]
        
    return x.reshape(-1, 1)

In [50]:
A = np.array([[2, 0, 0],
              [3, 1, 0],
              [6, 4, 2]])

b = np.array([1, 2, 3])

print(forward_substitution(A, b))

[[ 0.5]
 [ 0.5]
 [-1. ]]


The algorithm of the backward substitution is quite similar.

In [53]:
def backward_substitution(A, b):
    '''
    Backward Substitution
    Input: Upper triangular, full-ranked, squared matrix A, and the RHS b
    Output: Solution to this system x
    '''
    # Initialize and copy the identity
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)
    n = A.shape[0]
    m = A.shape[1]
    x = np.zeros(n)

    # Check if it is upper triangular, full-ranked, squared
    A_upper_tri = np.triu(A, k=0)
    if not(np.equal(A, A_upper_tri).all()) or (np.diag(A) == 0).all() or m != n:
        raise ValueError("The input matrix is not an upper triangular, full-ranked, squared matrix.")
    
    for i in range(n, 0, -1):
        i = i - 1
        for j in range(n - 1, i, -1):
            b[i] = b[i] - A[i, j] * x[j]
        x[i] = b[i] / A[i, i]

    return x.reshape(-1, 1)

In [54]:
A = np.array([
    [10, 2,  3],
    [0,  5,  6],
    [0,  0,  1]])

b = np.array([14, 11, 1])

print(backward_substitution(A, b))

[[0.9]
 [1. ]
 [1. ]]


## Gaussian Elimination
As shown before, it is easy to solve an upper or lower triangular matrix. Hence, we are seeking for a method to convert our linear system to such structure. Here, we introduced Gaussian Elimination. By doing multiple operations, we can convert a matrix to the upper triangular matrix. The opeation can be expressed by the matrix multiplication:

$$\textbf{U}=\textbf{M}_n\textbf{M}_{n-1}\cdots\textbf{M}_2\textbf{M}_1\textbf{A}$$

Where $\textbf{M}_i$ is some operations to the system. In Gaussian Elimination, we will have three types of operations: Permutation, Scaling, Elimination. We denote them as $\textbf{P}$, $\textbf{S}$ and $\textbf{E}$. We do the permutation for partial pivoting to avoid some small pivot numbers, and the reason for scaling is similar. Note that $\textbf{P}^{-1}=\textbf{P}^{T}$ and $\textbf{S}=\textbf{S}^{T}$ ($\textbf{S}$ is the diagonalized matrix). These two operations are all for **numerical stability**, which you will see later. But for now, let's talk about the elimination operation.

Every elimination operation can be expressed as a lower-triangular matrix where:

$$\textbf{E}_{ij}=\textbf{I}+c_{ij}\textbf{i}_i\textbf{i}_j^T,\quad i>j$$

$\textbf{E}_{ij}$ means we do row operation on row $i$ based on row $j$.

In [105]:
def gaussian_elimination(A, b):
    '''
    Gaussian Elimination
    Input: Full-ranked and squared matrix A, RHS: b
    Output: Solution x
    '''
    # Initialize and copy the identity
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)
    n = A.shape[0]
    m = A.shape[1]
    x = np.zeros(n)

    # Make sure to input squared matrix
    if m != n:
        raise ValueError("The input matrix is not a squared matrix.")
    
    for i in range(n):
        # Partial pivoting
        # It can be shown that partial pivoting will not affect the order of x
        idx = i + np.argmax(np.abs(A[i:, i]))
        if idx != i:
            # A
            row_temp = A[i, :].copy()
            A[i, :] = A[idx, :]
            A[idx, :] = row_temp
            # RHS
            b_temp = b[i].copy()
            b[i] = b[idx]
            b[idx] = b_temp

        for j in range(i + 1, n):
            num = A[j, i] / A[i, i]
            for k in range(i, n):
                A[j, k] = A[j, k] - A[i, k] * num
            b[j] = b[j] - b[i] * num

    x = backward_substitution(A, b)
    return x.reshape(-1, 1)

In [106]:
A = np.array([
    [2.0,  1.0, -1.0],
    [-3.0, -1.0,  2.0],
    [-2.0,  1.0,  2.0]])

b = np.array([-3.0, 5.0, 2.0])

print(gaussian_elimination(A, b))

[[ 1.]
 [-2.]
 [ 3.]]


## LU Decomposition
Recall the equation of Gaussian Elimination:

$$
\begin{align*}
\textbf{U} &= \textbf{M}_n\textbf{M}_{n-1}\cdots\textbf{M}_2\textbf{M}_1\textbf{A} \\
&= \textbf{E}_m\textbf{E}_{m-1}\cdots\textbf{E}_2\textbf{E}_1\textbf{P}\textbf{S}\textbf{A}
\end{align*}
$$

By doing some inversion, we can revert the upper-triangular matrix to the origin. For the scaling matrix $\textbf{S}$, since it is an diagonalized matrix, we do not need to pay much attention to it. For the permutation matrix $\textbf{P}$, We can just leave it with the original matrix $\textbf{A}$. Hence, the only thing we need to do is to inverse the elimination matrix $\textbf{E}$.

Since

$$\textbf{E}_{ij}=\textbf{I}+c_{ij}\textbf{i}_i\textbf{i}_j^T,\quad i>j$$

We can have its inverse

$$\textbf{E}_{ij}^{-1}=\textbf{I}-c_{ij}\textbf{i}_i\textbf{i}_j^T,\quad i>j$$

Note that $\textbf{E}_{ij}^{-1}$ is also lower-triangular matrix.

Then, we will find that

$$\underbrace{\textbf{E}^{-1}_{m}\textbf{E}^{-1}_{m-1}\cdots\textbf{E}^{-1}_{2}\textbf{E}^{-1}_{1}}_\textbf{L}\underbrace{\textbf{E}_m\textbf{E}_{m-1}\cdots\textbf{E}_2\textbf{E}_1\textbf{P}\textbf{S}\textbf{A}}_\textbf{U}=\textbf{PSA}$$

It is easy to figure out that the multiplication of lower-triangular matrices is also a lower-triangular matrix, then we got our LU factorization (Here, we choose to ignore $\textbf{S}$ for a better demostration.):

$$\textbf{LU}=\textbf{PA}$$

The most important thing is to understand **why we need LU Facorization**. By factorize $\textbf{A}$ to $\textbf{LU}$, we can reuse it in the future even if the RHS, $\textbf{b}$ is different.

In [121]:
def LU_decomposition(A):
    '''
    LU Decompostion
    Input:
    Full-ranked and squared matrix A
    Output:
    Decomposed lower and upper triangular matrix L and U
    Pivoting index vector p
    '''
    # Initialize and copy the identity
    U = np.array(A, dtype=float)
    n = U.shape[0]
    m = U.shape[1]
    L = np.eye(n)
    p = np.arange(n)

    # Make sure to input squared matrix
    if m != n:
        raise ValueError("The input matrix is not a squared matrix.")
    
    for i in range(n):
        # Partial pivoting
        # It can be shown that partial pivoting will not affect the order of x
        idx = i + np.argmax(np.abs(U[i:, i]))
        if idx != i:
            # U
            row_temp_U = U[i, :].copy()
            U[i, :] = U[idx, :]
            U[idx, :] = row_temp_U
            # L
            row_temp_L = L[i, :i].copy()
            L[i, :i] = L[idx, :i]
            L[idx, :i] = row_temp_L
            # p
            p[i], p[idx] = p[idx], p[i]
            
        # Here, we use a same algorithm as the Gaussian Elimination for U
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            for k in range(i, n):
                U[j, k] = U[j, k] - L[j, i] * U[i, k]
    
    return L, U, p

After performing the LU factorization, we can solve this system by using a forward substitution and a backward substitution:

$$
\begin{align*}
\textbf{PAx} &= \textbf{Pb} \\
\textbf{LUx} &= \textbf{Pb} \\
\end{align*}
\implies 
\begin{cases}
\textbf{Ly}=\textbf{Pb} \\
\textbf{Ux}=\textbf{y}
\end{cases}
$$

Doing a forward and backward substitution will cost much less than doing a gaussian elimination or something.

In [122]:
A = np.array([
    [2.0,  1.0, -1.0],
    [-3.0, -1.0,  2.0],
    [-2.0,  1.0,  2.0]])

b = np.array([-3.0, 5.0, 2.0])

L, U, p = LU_decomposition(A)

print("Lower triangular matrix:")
print(L)
print("Upper triangular matrix:")
print(U)

y = forward_substitution(L, b[p])
x = backward_substitution(U, y)

print("Solution:")
print(x)

Lower triangular matrix:
[[ 1.          0.          0.        ]
 [ 0.66666667  1.          0.        ]
 [-0.66666667  0.2         1.        ]]
Upper triangular matrix:
[[-3.         -1.          2.        ]
 [ 0.          1.66666667  0.66666667]
 [ 0.          0.          0.2       ]]
Solution:
[[ 1.]
 [-2.]
 [ 3.]]


## Gaussian-Jordan Elimination
*Will do this later, since didn't teach in MECH309*

## Summary of these Algorithms

| Algorithm | Leading Term (FLOPs) | Big-O Complexity | Best Use Case |
| :--- | :--- | :--- | :--- |
| **Forward / Backward Substitution** | $n^2$ | $O(n^2)$ | Solving triangular systems (after LU or QR). |
| **Gaussian Elimination** | $\frac{2}{3}n^3$ | $O(n^3)$ | General purpose solving for a single vector $b$. |
| **LU Factorization** | $\frac{2}{3}n^3$ | $O(n^3)$ | Efficient when solving $Ax = b$ for many different $b$ vectors. |
| **Gauss-Jordan Elimination** | $n^3$ | $O(n^3)$ | Finding the matrix inverse (rarely recommended for solving systems). |